In [4]:
import pandas as pd 
df = pd.read_csv('census_puma_nyc_2011-2017.csv').drop('Unnamed: 0', axis =1)

In [8]:
from sklearn.ensemble import RandomForestRegressor # classifier imputation 
from sklearn.metrics import r2_score
from IPython.display import clear_output
list_of_groups = []
outer_counter = 0
accuracies = []
cols_to_not_include = ['mergeid','state','county','tract','zcta(zip)','puma'] 
df_group = df 
groups_nulls = df_group.columns[df_group.isnull().any()].drop(['mergeid','BORO NUM','puma','zcta(zip)', 'Census Tract', 'YEAR','conceverted census tract'], errors = 'ignore')
not_nas_df  = df_group[df_group.columns[~df_group.isnull().any()]].drop(['mergeid','state','puma','county','tract','zcta(zip)','county_','converted_tract'], axis = 1,errors = 'ignore')
inner_counter = 0

for cols in groups_nulls:
    cols_na = df_group[cols]
    if cols_na.isna().sum() == len(cols_na):
        # skip for now 
        continue 

    dropped_na = cols_na.dropna()
    nas_rows = cols_na[cols_na.isna()].index
    to_predict = not_nas_df.loc[nas_rows]
    X = not_nas_df.loc[dropped_na.index]    
    Y = dropped_na 
    lm = RandomForestRegressor(n_estimators = 300,n_jobs = -1).fit(X,Y) # another technique 
    predict_forcolumn =  lm.predict(to_predict)
    predict_training_data = lm.predict(X)
    training_data_accuracy = r2_score(Y,predict_training_data)
    accuracies.append(training_data_accuracy)
    cols_na.loc[cols_na[cols_na.isna()].index] = predict_forcolumn
    df_group[cols] = cols_na
    not_nas_df  = df_group[df_group.columns[~df_group.isnull().any()]].drop(['mergeid','state','county','tract','zcta(zip)','puma'], axis = 1,errors = 'ignore')
    print("inner %")
    print(inner_counter / len(groups_nulls) * 100)
    print(training_data_accuracy)
    inner_counter += 1 
    clear_output(wait = True)
outer_counter += 1 


list_of_groups.append(df_group) 

inner %
50.0
0.9922732610123431


C:\Users\Voltaire Vergara\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
df_group.to_csv('data_imputated.csv',index = False)

In [3]:
import geopandas as gpd
gdf_community = gpd.read_file('evictions_zcta_shapefiles/community_district_shp/nycd_20d/nycd.shp')
borough = {'1':'Manhattan','2':'Bronx','3':'Brooklyn','4':'Queens','5':'Staten Island'}

In [18]:
gdf_community['Borough'] = gdf_community['BoroCD'].astype(str).str[0].map(borough)

In [19]:
gdf_community['coords'] = gdf_community['geometry'].apply(lambda x: x.representative_point().coords[:])
gdf_community['coords'] = [coords[0] for coords in gdf_community['coords']]

In [14]:
import zipfile 
from io import BytesIO
import geopandas as gpd
from urllib.request import urlopen
import json
with urlopen('https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nycd_21a.zip') as response:
    file = zipfile.ZipFile(BytesIO(response.read()))
    file.extractall('community_districts_shapefiles/')

In [15]:
gdf = gpd.read_file('community_districts_shapefiles/nycd_21a/nycd.shp')
borough = {'1':'Manhattan','2':'Bronx','3':'Brooklyn','4':'Queens','5':'Staten Island'}

In [28]:
cds_dict = {
'Bronx' :  {1:3710,
            2:3710,
            3:3705, 
            6:3705, 
            4:3708, 
            5:3707, 
            7:3706, 
            8:3701, 
            9:3709, 
            10:3703, 
            11:3704, 
            12:3702}, 
'Manhattan': {
            1:3810,  
            2:3810, 
            3:3809, 
            4:3807,
            5:3807, 
            6:3808, 
            7:3806, 
            8:3805, 
            9:3802, 
            10:3803, 
            11:3804,
            12:3801},
'Staten Island': {
            1:3903, 
            2:3902, 
            3:3901},
'Brooklyn': {
            1:4001, 
            2:4004, 
            3:4003, 
            4:4002, 
            5:4008, 
            6:4005, 
            7:4012, 
            8:4006, 
            9:4011, 
            10:4013,
            11:4017, 
            12:4014, 
            13:4018, 
            14:4015, 
            15:4016, 
            16:4007,
            17:4010, 
            18:4009},
'Queens':  {  
            1:4101, 
            2:4109, 
            3:4102,
            4:4107, 
            5:4110,
            6:4108,
            7:4103, 
            8:4106, 
            9:4111, 
            10:4113, 
            11:4104,
            12:4112,
            13:4105,
            14:4114}}


{1: 3710,
 2: 3710,
 3: 3705,
 6: 3705,
 4: 3708,
 5: 3707,
 7: 3706,
 8: 3701,
 9: 3709,
 10: 3703,
 11: 3704,
 12: 3702}

In [34]:
gdf_community['CD'] = gdf_community['BoroCD'].astype(str).str[1:].astype(int)

In [42]:
import numpy as np 
def apply_cd(x):
    try:
        return cds_dict[x['Borough']][x['CD']]
    except:
        return np.nan
gdf_community['PUMA'] = gdf_community.apply(lambda x: apply_cd(x), axis = 1)

In [43]:
gdf_community

,BoroCD,Shape_Leng,Shape_Area,geometry,Borough,coords,CD,PUMA
0,101,74168.587892,4.268912e+07,"MULTIPOLYGON (((972081.788 190733.467, 972184....",Manhattan,"(981880.1073585525, 199110.52139282227)",1,3810.0
1,102,34130.595277,3.768921e+07,"POLYGON ((985125.733 207917.071, 986018.960 20...",Manhattan,"(983540.3175902284, 204992.4183959961)",2,3810.0
2,103,30695.640963,4.687978e+07,"POLYGON ((992073.467 203714.076, 992068.667 20...",Manhattan,"(988898.8268376964, 202088.3316040188)",3,3809.0
3,104,67623.669916,4.931172e+07,"POLYGON ((985929.324 220967.002, 985995.954 22...",Manhattan,"(984623.4121845987, 214645.5505065918)",4,3807.0
4,105,35288.188527,4.379033e+07,"POLYGON ((991725.244 217725.299, 992169.505 21...",Manhattan,"(987706.0399258747, 212856.5712890625)",5,3807.0
...,...,...,...,...,...,...,...,...
66,484,244366.697480,1.213293e+08,"MULTIPOLYGON (((1003211.277 142534.469, 100337...",Queens,"(1017659.2868182368, 145305.13150024414)",84,NaN
67,501,153873.506331,3.772536e+08,"MULTIPOLYGON (((939997.095 173013.579, 939915....",Staten Island,"(949466.8889466145, 167160.7006225586)",1,3903.0
68,502,143044.989882,5.927157e+08,"MULTIPOLYGON (((963842.054 150006.498, 963845....",Staten Island,"(948139.8837331245, 156940.55541992188)",2,3902.0
69,503,193564.186363,5.990534e+08,"MULTIPOLYGON (((947226.690 134695.710, 947277....",Staten Island,"(930519.6951070898, 135391.6058044508)",3,3901.0
